[Correct way to try/except using Python requests module?](https://stackoverflow.com/questions/16511337/correct-way-to-try-except-using-python-requests-module)

# 新北車位24h改良

In [1]:
import time
import requests
from time import gmtime, strftime

In [ ]:
url = 'https://data.ntpc.gov.tw/od/data/api/1A71BA9C-EF21-4524-B882-6D085DF2877A?$format=csv&$top=2000&$skip=0'

## Build the request function

In [2]:
def do_request(url):
    try:
#         response = {'success': False, 'err': None, 'content': None}
        r = requests.get(url, allow_redirects=True)
        r.raise_for_status() # 看有沒有錯誤
        if len(r.text) == 0:
            return {'success': False, 'err': 'Null content', 'content': None}
        # 成功才回傳 request 內容
        return {'success': True, 'err': None, 'content': r}
        
    except requests.exceptions.Timeout:
        # Maybe set up for a retry, or continue in a retry loop
        return {'success': False, 'err': 'Timeout', 'content': None}
    except requests.exceptions.TooManyRedirects:
        # Tell the user their URL was bad and try a different one
        return {'success': False, 'err': 'Too Many Redirects', 'content': None}
    except requests.exceptions.RequestException as e:
         # catastrophic error. bail.
        return {'success': False, 'err': 'Request Exception', 'content': None}

In [ ]:
# 0~15, total 16 次
for i in range(16): 
    n = 0 + i*2000
    url = "https://data.ntpc.gov.tw/od/data/api/1A71BA9C-EF21-4524-B882-6D085DF2877A?$format=json&$top=2000&$skip="+str(n)
    res = do_request(url)
    print(i+1, res['err'])

## Call the function to download WHOLE file

- https://stackoverflow.com/questions/23615496/removing-the-first-line-of-csv-file
- https://stackoverflow.com/questions/30833409/python-deleting-the-first-2-lines-of-a-string

In [3]:
def do_write(content, filename, i):
    with open(filename, 'a', encoding='utf-8') as f:
        if i == 0:
            # write title once in begining
            f.write(content)
        else:
            # skip header line
            postString = content.split("\n",2)[1:]
            for line in postString:
                f.write(line)

In [4]:
def job():
    # Get time for file name
    time.sleep(5)
    ctime = time.localtime(time.mktime(time.localtime()))
    filename = strftime("%Y-%m-%d %H-%M-all", ctime) + '.csv'
    
    # Looping to download, i: 0~15
    for i in range(16): 
        n = 0 + i*2000
        url = "https://data.ntpc.gov.tw/od/data/api/1A71BA9C-EF21-4524-B882-6D085DF2877A?$format=csv&$top=2000&$skip="+str(n)
        res = do_request(url)
        time.sleep(1)
        
        # Validation
        tries = 0
        while tries < 3:

            # Response SUCCESS > write file > break validation loop
            if res['success'] == True:
                do_write(content = res['content'].text, 
                         filename= filename, 
                         i = i)
                break
                            
            # Response FAIL > redo request
            else:
                # Redo request
                tries += 1
                time.sleep(30)
                print(f'redo at {i} #{filename}' + ", Error: " + res['err'])
                res = do_request(url)
                if res['success'] == True:
                    do_write(content = res['content'].text, 
                             filename= filename, 
                             i = i)
                    break
        
#         if res['success'] == True and tries < 3:
#             with open(filename, 'a', encoding='utf-8') as f:
#                 # write title once in begining
#                 if i == 0:
#                     f.write(res['content'].text)
#                 else:
#                     content = res['content'].text
#                     next(content) # skip header line
#                     for line in content:
#                         f.write(line)
                
#         else:
#             # Redo request
#             tries+=1
#             time.sleep(30)
#             res = do_request(url)
            

In [ ]:
job()

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
sched = BlockingScheduler()

sched.add_job(job, 'cron', hour='*', minute='0,10,20,30,40,50')
sched.start()

redo at 2 #2020-05-14 09-50-all.csv, Error: Null content
redo at 1 #2020-05-14 10-00-all.csv, Error: Null content
redo at 2 #2020-05-14 10-10-all.csv, Error: Null content
redo at 1 #2020-05-14 10-20-all.csv, Error: Null content
redo at 1 #2020-05-14 10-30-all.csv, Error: Null content
redo at 3 #2020-05-14 10-40-all.csv, Error: Null content
redo at 0 #2020-05-14 10-50-all.csv, Error: Null content
redo at 0 #2020-05-14 11-00-all.csv, Error: Null content
redo at 4 #2020-05-14 11-10-all.csv, Error: Null content
redo at 0 #2020-05-14 11-20-all.csv, Error: Null content
redo at 2 #2020-05-14 11-30-all.csv, Error: Null content
redo at 1 #2020-05-14 11-40-all.csv, Error: Null content
redo at 0 #2020-05-14 11-50-all.csv, Error: Null content
redo at 0 #2020-05-14 12-00-all.csv, Error: Null content
redo at 1 #2020-05-14 12-10-all.csv, Error: Null content
redo at 0 #2020-05-14 12-20-all.csv, Error: Null content
redo at 0 #2020-05-14 12-30-all.csv, Error: Null content
redo at 0 #2020-05-14 12-40-all